In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
from google.colab import files
import zipfile, os

print("\nPlease upload your dataset ZIP file")
uploaded = files.upload()

for file_name in uploaded.keys():
    zip_path = file_name
    print(f"Unzipping {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("dataset-resized")

train_dir = "/content/dataset-resized"
print("Dataset extracted to:", train_dir)



In [ ]:
!mv dataset-resized/dataset-resized/* dataset-resized/
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)
model.save("waste_classifier_model.h5")
print("\nModel saved as waste_classifier_model.h5")
print("\n📸 Upload an image to test the model")

uploaded = files.upload()
class_labels = list(train_generator.class_indices.keys())

for file_name in uploaded.keys():
    img_path = file_name
    img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)

    print(f"\nPredicted Waste Type for {file_name}: {class_labels[predicted_class[0]]}")

Found 2022 images belonging to 6 classes.
Found 503 images belonging to 6 classes.

✅ Classes found: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,734 (42.61 MB)

 Trainable params: 11,169,734 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 263s 4s/step - accuracy: 0.2460 - loss: 2.2701 - val_accuracy: 0.2843 - val_loss: 1.7072
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 261s 4s/step - accuracy: 0.3884 - loss: 1.4898 - val_accuracy: 0.2922 - val_loss: 1.6208
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 278s 4s/step - accuracy: 0.4033 - loss: 1.4655 - val_accuracy: 0.3738 - val_loss: 1.4538
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 259s 4s/step - accuracy: 0.4813 - loss: 1.2924 - val_accuracy: 0.3678 - val_loss: 1.5262
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 258s 4s/step - accuracy: 0.4929 - loss: 1.3108 - val_accuracy: 0.4473 - val_loss: 1.3373
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 259s 4s/step - accuracy: 0.5248 - loss: 1.2452 - val_accuracy: 0.4493 - val_loss: 1.4259
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 261s 4s/step - accuracy: 0.5649 - loss: 1.1324 - val_accuracy: 0.4791 - val_loss: 1.3736
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 259s 4s/step - accuracy: 0.5922 - loss: 1.1104 - val_accuracy: 0.4115 - v


✅ Model saved as waste_classifier_model.h5

📸 Upload an image to test the model


Saving plastic7.jpg to plastic7.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step

♻️ Predicted Waste Type for plastic7.jpg: glass
